In [28]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

In [29]:
hard21 = pd.read_csv("2021hard.csv")
clay21 = pd.read_csv("2021clay.csv")
grass21 = pd.read_csv("2021grass.csv")

hard22 = pd.read_csv("2022hard.csv")
clay22 = pd.read_csv("2022clay.csv")
grass22 = pd.read_csv("2022grass.csv")

hard23 = pd.read_csv("2023hard.csv")
clay23 = pd.read_csv("2023clay.csv")
grass23 = pd.read_csv("2023grass.csv")
overall23 = pd.read_csv("2023overall.csv")

hard = [hard21, hard22, hard23]
clay = [clay21, clay22, clay23]
grass = [grass21, grass22, grass23]

In [30]:
#Renaming the columns
hard = [df.rename(columns={"value": "Hard_Win%"}) for df in hard]
clay = [df.rename(columns={"value": "Clay_Win%"}) for df in clay]
grass = [df.rename(columns={"value": "Grass_Win%"}) for df in grass]

overall23 = overall23.rename(columns = {"value": "Overall_Win%"})
hard21 = hard[0]
hard22 = hard[1]
hard23 = hard[2]

clay21 = clay[0]
clay22 = clay[1]
clay23 = clay[2]

#lets just focus on 2023
grass21 = grass[0]
grass22 = grass[1]
grass23 = grass[2]

In [31]:
hard23.head()

,rank,name,country_name,country_id,Hard_Win%
0,1,Novak Djokovic,Serbia,SRB,96.43%
1,2,Daniil Medvedev,Russian Federation,RUS,83.93%
2,3,Jannik Sinner,Italy,ITA,83.33%
3,4,Carlos Alcaraz Garfia,Spain,ESP,81.25%
4,5,Taylor Harry Fritz,United States,USA,74.51%


In [33]:
#Now, let's merge all the 21 dfs together?
conn = sqlite3.connect("tennis.db")
hard23.to_sql("hard", conn, if_exists = 'append', index = False)
clay23.to_sql("clay", conn, if_exists = 'append', index = False)
grass23.to_sql("grass", conn, if_exists = 'append', index = False)


OperationalError: table grass has no column named Grass_Win%

In [ ]:
cursor = conn.cursor()

#combine all the names maybe?

In [ ]:
conn = sqlite3.connect("tennis.db")
cursor = conn.cursor()

#sql query
cmd = \
""" 
SELECT H.name, H.country_name, H.'Hard_Win%', C.'Clay_Win%', G.'Grass_Win%'
FROM hard H
INNER JOIN clay C ON H.name = C.name
INNER JOIN grass G on H.name = G.name;
"""
tentative = pd.read_sql_query(cmd, conn) #read the query to a dataframe
conn.close()
tentative

DatabaseError: Execution failed on sql ' 
SELECT H.name, H.country_name, H.'Hard_Win%', C.'Clay_Win%', G.'Grass_Win%'
FROM hard H
INNER JOIN clay C ON H.name = C.name
INNER JOIN grass G on H.name = G.name;
': no such column: G.Grass_Win%

In [ ]:
#for tennisabstract beautifulsoup, let's use the td tags